## Clean up your data
The beer data collected includes beers with atleast 1 review from the users.
We build a dataset (final_data.csv) of only beer and users with at least 10 reviews.
The algorithm to be used to implement the recommendation system is item-based CF.

**Collaborative filtering (CF)** is divided into nearest neighbor collaborative filtering and latent factor collaborative filtering.

And the nearest neighbor collaborative filtering is again divided into user-based and item-based.

Among them, the reasons for choosing item-based are as follows.

1.  The individual tastes of users are too diverse.
2.  On the other hand, the number of beer users leaving a review is limited.

Therefore, we decided that it is appropriate to recommend it on an item basis.

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

import warnings

warnings.filterwarnings('ignore')

In [2]:
# User who left more than n data reviews, a function to filter beer
def preprocessing(data, n):
    min_id = data['user'].value_counts() >= n
    min_id = min_id[min_id].index.to_list()
    data = data[data['user'].isin(min_id)]

    min_beer = data['beer_name'].value_counts() >= n
    min_beer = min_beer[min_beer].index.to_list()
    data = data[data['beer_name'].isin(min_beer)]

    return data

In [3]:
tmp = pd.read_csv("final_data.csv",encoding= 'unicode_escape')
temp=tmp.copy()

# Repeat 10 times.
for i in range(1,10):
    temp = preprocessing(temp, 10)
    print(temp.shape)

temp.to_csv('Refined_data.csv', encoding='utf-8')

(57022, 10)
(57013, 10)
(57013, 10)
(57013, 10)
(57013, 10)
(57013, 10)
(57013, 10)
(57013, 10)
(57013, 10)


## The existing data was about 91,776, but it is now reduced to 57,013.

## **Recommendation based on similarity between beer**

### We will now recommend similar beers based on the similarity between beers.

The criterion for measuring similarity was **Cosine similarity** and **Pearson similarity**.

Pearson similarity is used when too large or too small a value significantly affects the similarity.

There was no outlier in the EDA process.

We confirmed that the ratings also follow a normal distribution.

So we want to use cosine similarity.

Among the refined data, the matrix is ​​composed of 3 columns necessary for the implementation of the recommendation system.

The three factors are **User, Beer Name, and Rating.**

In [5]:
data = pd.read_csv('Refined_data.csv', encoding='utf-8', index_col=0)

ratings = data.copy()

# Construct a user-ID matrix using a pivot table
ratings_matrix = ratings.pivot_table('rating', index='user', columns='beer_name')
ratings_matrix.head(3)
# Nan processing using fillna function
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

beer_name,Hoegaarden Rosée,?ubr,Alexander Keith's India Pale Ale,Baltika 7 Eksportnoe (Export),Bavaria 8.6 (Original),Beck's,Bellwoods Bring Out Your Dead,Bellwoods Jelly King (Raspberry and Blackberry),Berthold Keller Premium Lager,Boxer Ice,BrewDog / Weihenstephan India Pale Weizen,Bud Light,Bud Light Apple,Bud Light Lime,Budweiser,Cantillon Vin Jaune,Carlsberg Pilsner,Chang Beer,Cool Beer Blonde Lager,Corona Extra,Creemore Springs Premium Lager,DAB Original,Dieu du Ciel! Péché Mortel (Bourbon),Dos Equis XX Special Lager,Dragon Stout,Driftwood Fat Tug IPA,Driftwood Sartori Harvest IPA,Erdinger Weissbier,Estrella Damm (5.4%),Flying Monkeys 12 Minutes to Destiny,Four Winds Nectarous,Great Lakes Brewery Canuck Pale Ale,Guinness Draught,Harp Lager,Heineken,Hoegaarden Grand Cru,Innis & Gunn The Original,Kirin Ichiban,Kona Big Wave Golden Ale,Kozel Cerný (Dark),...,MacKinnon Brothers Red Fox,Maclay's Traditional Pale Ale,Mad Jack Apple Lager,Maisel & Friends Marc's Chocolate Bock,Michelob Ultra,Mickeys Fine Malt Liquor,Midnight Sun M,Mill Street Original Organic Lager,Miller Lite,Molson Carling Light,Moosehead Pale Ale,Muskoka Cream Ale,Muskoka Mad Tom IPA,Newcastle Brown Ale (Non-US Version),Nickel Brook Glory & Gold,NÃ¤rke Kaggen Stormaktsporter,Old Milwaukee,Paulaner Oktoberfest Bier (Wiesn Bier),Peroni,Pilsner Urquell,Rickard's Red,Samuel Smiths Organic Lager,San Miguel,Sapporo Premium Beer / Draft Beer,Shawn & Ed Lagershed Original,Sleeman Clear 2.0,Smithwick's Ale / Draught,Steam Whistle Pilsner,Stella Artois,Superstition Berry White,Tatra Jasne Pe?ne / Pils / Beer,Tiger Beer,Toppling Goliath Kentucky Brunch,Tsingtao Beer Quality Series (Taiwan),Unibroue La Fin du Monde,Unibroue Trois Pistoles,Walkerville Classic Amber Lager,Waterloo IPA,Weihenstephaner Hefeweissbier,Wellington Special Pale Ale SPA
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.0,0.0,0.0,0.0,0.0,2.6,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,2.0,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,3.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7,2.3,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,4.2,0.0,0.0,0.0,0.0,0.0,3.1,0.0,0.0,0.0,2.6,0.0,0.0,0.0,0.0,2.9,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
"00cobraR(1,103)",0.0,0.0,1.5,0.0,0.0,2.3,0.0,0.0,0.0,0.0,0.0,1.1,0.0,0.8,1.4,0.0,0.0,0.0,0.0,1.4,0.0,0.0,0.0,2.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4,0.0,2.4,0.0,0.0,0.0,0.0,0.0,...,0.0,2.4,0.0,0.0,0.8,0.0,0.0,0.0,1.1,0.0,0.0,0.0,0.0,1.9,0.0,0.0,0.6,0.0,0.0,2.6,2.9,0.0,0.0,1.6,0.0,0.0,0.0,0.0,1.9,0.0,0.0,0.0,0.0,0.0,4.0,4.2,0.0,0.0,3.7,0.0
0o9i(439),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,2.1,0.0,0.0,0.0,0.0,0.0,2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,2.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2,0.0,0.0,0.0,0.0,0.0,3.9,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.8,0.0
1010(309),0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.5,0.0,0.0,0.0,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,2.1,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.1,0.0,2.1,0.0,0.0,0.0,0.0,0.0,4.1,0.0,0.0,0.0,3.7,0.0
"11026(1,772)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.8,2.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.8,0.0,0.0,0.0,0.0,3.1,0.0,0.0,0.0,0.0,0.0,2.8,2.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.6,0.0,0.0,1.0,0.0,0.0,4.1,3.9,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"zebracakes(1,339)",0.0,0.0,0.0,0.8,0.0,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0

In [6]:
# Transpose for similarity calculation
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T

user,007lund(92),"00cobraR(1,103)",0o9i(439),1010(309),"11026(1,772)",12HundredBaud(262),"17Beer(1,323)","17thfloor(2,444)","17wojownik(1,326)",184601(206),"1FastSTi(3,361)",1guywith_aboat(254),20107589(748),"21iceman40(1,735)","3fourths(9,493)",40oz(181),"5000(7,050)",502Che(999),"574deadzone(1,420)","60minuteman(1,764)",666seth(229),"77ships(14,200)",AAEA(266),"ABUSEDGOAT(3,816)",ADfan(870),AGAquino(669),"ALIBALI(4,265)","ALLOVATE(3,542)",AMN(127),"ANZUShophunter(1,828)",AOF(306),"AR(1,002)","ARH(1,141)",ARMed(625),ATOmemphis(200),AWMiller(512),"AWS9(1,054)",Aarleks(409),Aaron68(608),AaronBouchard(150),...,winkle(478),"wlajwl(10,003)",wnhay(722),wnp22(698),"wombat23(3,348)","worldbeertaste(1,845)","wozap(1,776)",wpp83(64),wulfhere(218),"wunderbier(1,434)","wxman(2,024)","wyzzywyz22(2,108)",xmandax(130),"xnoxhatex(1,305)",y0cola(361),yalnikim(800),yarry(528),yayforbeer(753),yerffoeg(669),"yespr(54,703)","yesyouam(1,016)","yiannnos(1,254)","yngwie(19,386)","yobdoog(1,888)",yrstruly(359),"yy0pp(1,664)","z87(2,591)","zacgillbanks(7,429)","zach8270(4,021)",zakkbironia(112),zappafan99(463),"zathrus13(1,776)",zborgerd(187),zdet444dd(440),"zdk(1,917)","zebracakes(1,339)","zimbo(1,018)",zombywoof(486),"zvikar(11,342)","zvsn(5,611)"
beer_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Hoegaarden Rosée,0.0,0.0,0.0,0.0,0.0,3.4,0.0,0.0,0.0,0.0,0.0,0.0,2.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.4,0.0,0.0,0.0,0.0,2.5,3.3,0.0,0.0,1.9,3.0,0.0,2.5,2.6,2.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.8
?ubr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.8,0.0,...,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,0.0,0.0,2.1,0.0,0.0,0.0,1.8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,0.0
Alexander Keith's India Pale Ale,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.2,0.0,0.0,0.0,2.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.8,0.0,0.0,2.7,0.0,0.0,2.3,0.0,0.0,0.0,0.0,0.0,2.8,0.0,0.0,0.0,0.0,0.0,1.7,0.0,0.0,0.0,0.0,0.0
Baltika 7 Eksportnoe (Export),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.6,0.0,0.0,0.0,0.0,2.3,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,2.1,1.8,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.9,0.0,1.2,2.2,0.0,0.0,0.0,0.0,1.6,2.7,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,2.2,0.0
Bavaria 8.6 (Original),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.7,0.0,0.0,0.0,0.0,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,2.1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Unibroue Trois Pistoles,0.0,4.2,0.0,0.0,3.9,0.0,3.4,4.1,0.0,0.0,3.9,4.2,0.0,4.1,4.0,4.1,4.0,3.2,4.1,0.0,0.0,4.0,0.0,3.7,0.0,3.9,0.0,2.8,0.0,0.0,0.0,4.9,3.6,4.4,3.8,4.4,0.0,0.0,0.0,0.0,...,4.5,4.6,4.3,4.5,0.0,0.0,0.0,0.0,3.9,4.2,3.4,0.0,0.0,3.8,0.0,3.5,3.5,4.2,0.0,3.3,4.2,0.0,4.1,0.0,0.0,0.0,0.0,0.0,3.9,0.0,2.1,3.8,0.0,4.4,3.8,3.7,0.0,4.4,3.0,0.0
Walkerville Classic Amber Lager,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.8,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [7]:
# Finding cosine similarity from item-user matrix
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# Convert the beer name to a DataFrame by mapping the beer name to the NumPy matrix returned by cosine_similarity()
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(82, 82)


beer_name,Hoegaarden Rosée,?ubr,Alexander Keith's India Pale Ale,Baltika 7 Eksportnoe (Export),Bavaria 8.6 (Original),Beck's,Bellwoods Bring Out Your Dead,Bellwoods Jelly King (Raspberry and Blackberry),Berthold Keller Premium Lager,Boxer Ice,BrewDog / Weihenstephan India Pale Weizen,Bud Light,Bud Light Apple,Bud Light Lime,Budweiser,Cantillon Vin Jaune,Carlsberg Pilsner,Chang Beer,Cool Beer Blonde Lager,Corona Extra,Creemore Springs Premium Lager,DAB Original,Dieu du Ciel! Péché Mortel (Bourbon),Dos Equis XX Special Lager,Dragon Stout,Driftwood Fat Tug IPA,Driftwood Sartori Harvest IPA,Erdinger Weissbier,Estrella Damm (5.4%),Flying Monkeys 12 Minutes to Destiny,Four Winds Nectarous,Great Lakes Brewery Canuck Pale Ale,Guinness Draught,Harp Lager,Heineken,Hoegaarden Grand Cru,Innis & Gunn The Original,Kirin Ichiban,Kona Big Wave Golden Ale,Kozel Cerný (Dark),...,MacKinnon Brothers Red Fox,Maclay's Traditional Pale Ale,Mad Jack Apple Lager,Maisel & Friends Marc's Chocolate Bock,Michelob Ultra,Mickeys Fine Malt Liquor,Midnight Sun M,Mill Street Original Organic Lager,Miller Lite,Molson Carling Light,Moosehead Pale Ale,Muskoka Cream Ale,Muskoka Mad Tom IPA,Newcastle Brown Ale (Non-US Version),Nickel Brook Glory & Gold,NÃ¤rke Kaggen Stormaktsporter,Old Milwaukee,Paulaner Oktoberfest Bier (Wiesn Bier),Peroni,Pilsner Urquell,Rickard's Red,Samuel Smiths Organic Lager,San Miguel,Sapporo Premium Beer / Draft Beer,Shawn & Ed Lagershed Original,Sleeman Clear 2.0,Smithwick's Ale / Draught,Steam Whistle Pilsner,Stella Artois,Superstition Berry White,Tatra Jasne Pe?ne / Pils / Beer,Tiger Beer,Toppling Goliath Kentucky Brunch,Tsingtao Beer Quality Series (Taiwan),Unibroue La Fin du Monde,Unibroue Trois Pistoles,Walkerville Classic Amber Lager,Waterloo IPA,Weihenstephaner Hefeweissbier,Wellington Special Pale Ale SPA
beer_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Hoegaarden Rosée,1.000000,0.319297,0.141732,0.385892,0.303081,0.346399,0.157306,0.033456,0.170984,0.031929,0.284598,0.208556,0.093603,0.140874,0.324163,0.124544,0.371421,0.385801,0.046788,0.334569,0.131876,0.252795,0.215070,0.229090,0.287847,0.155861,0.065133,0.400915,0.362408,0.095571,0.136411,0.140805,0.355992,0.156334,0.328475,0.448265,0.404792,0.349582,0.334326,0.435032,...,0.021186,0.058505,0.053945,0.355109,0.167196,0.110505,0.092425,0.132623,0.154030,0.000000,0.213925,0.119132,0.148944,0.319935,0.073273,0.338745,0.139386,0.379338,0.271379,0.363834,0.127679,0.173075,0.174783,0.283818,0.029268,0.068788,0.178523,0.124118,0.337631,0.126474,0.346741,0.384780,0.113703,0.059851,0.240113,0.231277,0.009895,0.090663,0.385191,0.077389
?ubr,0.319297,1.000000,0.228373,0.373280,0.343121,0.364124,0.154115,0.086864,0.267956,0.113118,0.244281,0.207803,0.069815,0.129006,0.299224,0.056968,0.337592,0.301987,0.246627,0.345950,0.239077,0.342705,0.210481,0.240851,0.332217,0.210297,0.107825,0.395508,0.343261,0.167358,0.167141,0.206864,0.345215,0.181726,0.347578,0.385596,0.394237,0.346643,0.263277,0.374410,...,0.066357,0.250535,0.072997,0.311773,0.181601,0.139911,0.064990,0.227889,0.155549,0.135593,0.284227,0.259791,0.235454,0.291805,0.049029,0.234694,0.178762,0.337959,0.266276,0.368616,0.208910,0.184146,0.114449,0.310334,0.090207,0.186664,0.206769,0.267545,0.326621,0.050311,0.502963,0.413082,0.071996,0.037117,0.257388,0.259893,0.193497,0.177898,0.338697,0.268749
Alexander Keith's India Pale Ale,0.141732,0.228373,1.000000,0.247895,0.169166,0.318291,0.182993,0.130136,0.249939,0.134634,0.061400,0.313484,0.129463,0.270114,0.318094,0.037856,0.308892,0.187796,0.220220,0.310131,0.463718,0.314224,0.222315,0.297026,0.332077,0.320893,0.174642,0.338452,0.262418,0.206025,0.215029,0.337443,0.355844,0.330843,0.345178,0.180118,0.337364,0.278045,0.235173,0.156546,...,0.115333,0.342045,0.131283,0.061361,0.275727,0.223262,0.113185,0.414599,0.264485,0.104852,0.250289,0.371072,0.362283,0.344748,0.097334,0.167241,0.250427,0.216305,0.223505,0.339026,0.545451,0.221877,0.062208,0.3

In [8]:
# Calculated Cosine similarity above and we found that there are 82 types of beers.
ratings_matrix.columns

Index([' Hoegaarden Rosée', '?ubr', 'Alexander Keith's India Pale Ale',
       'Baltika 7 Eksportnoe (Export)', 'Bavaria 8.6 (Original)', 'Beck's',
       'Bellwoods Bring Out Your Dead',
       'Bellwoods Jelly King (Raspberry and Blackberry)',
       'Berthold Keller Premium Lager', 'Boxer Ice',
       'BrewDog / Weihenstephan India Pale Weizen', 'Bud Light',
       'Bud Light Apple', 'Bud Light Lime', 'Budweiser', 'Cantillon Vin Jaune',
       'Carlsberg Pilsner', 'Chang Beer', 'Cool Beer Blonde Lager',
       'Corona Extra', 'Creemore Springs Premium Lager', 'DAB Original',
       'Dieu du Ciel! Péché Mortel (Bourbon)', 'Dos Equis XX Special Lager',
       'Dragon Stout', 'Driftwood Fat Tug IPA',
       'Driftwood Sartori Harvest IPA', 'Erdinger Weissbier',
       'Estrella Damm (5.4%)', 'Flying Monkeys 12 Minutes to Destiny',
       'Four Winds Nectarous', 'Great Lakes Brewery Canuck Pale Ale',
       'Guinness Draught', 'Harp Lager', 'Heineken', 'Hoegaarden Grand Cru',
       'In

In [9]:
# Extracting only 5 beers with a similarity to Kozel Beer
item_sim_df['Kozel Cerný (Dark)'].sort_values(ascending=False)[:5]

beer_name
Kozel Cerný (Dark)                        1.000000
Pilsner Urquell                           0.496766
Erdinger Weissbier                        0.493613
Hoegaarden Grand Cru                      0.485263
Paulaner Oktoberfest Bier (Wiesn Bier)    0.477303
Name: Kozel Cerný (Dark), dtype: float64

In [10]:
# Extracting only 5 beers with a similarity to Hoegaarden Beer
item_sim_df['Hoegaarden Grand Cru'].sort_values(ascending=False)[:5]

beer_name
Hoegaarden Grand Cru             1.000000
Erdinger Weissbier               0.523739
Kozel Cerný (Dark)               0.485263
Weihenstephaner Hefeweissbier    0.471089
Pilsner Urquell                  0.468508
Name: Hoegaarden Grand Cru, dtype: float64

## **Personalized beer recommendations**

### Now we will implement a recommendation system that reflects the individual's rating.

In [12]:
# ratings_arr.dot(item_sim_arr) is the rating * beer similarity
# ratings_arr is the actual rating vector for the Top_N items with the highest similarity to item i of user u.
# item_sim_arr is the similarity vector of Top_N items with the highest similarity to item i
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

# Finding a personalized prediction score
# Only the rating value and the similarity value are extracted and assigned
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns = ratings_matrix.columns)

# Predicted score calculated for each individual
ratings_pred_matrix

beer_name,Hoegaarden Rosée,?ubr,Alexander Keith's India Pale Ale,Baltika 7 Eksportnoe (Export),Bavaria 8.6 (Original),Beck's,Bellwoods Bring Out Your Dead,Bellwoods Jelly King (Raspberry and Blackberry),Berthold Keller Premium Lager,Boxer Ice,BrewDog / Weihenstephan India Pale Weizen,Bud Light,Bud Light Apple,Bud Light Lime,Budweiser,Cantillon Vin Jaune,Carlsberg Pilsner,Chang Beer,Cool Beer Blonde Lager,Corona Extra,Creemore Springs Premium Lager,DAB Original,Dieu du Ciel! Péché Mortel (Bourbon),Dos Equis XX Special Lager,Dragon Stout,Driftwood Fat Tug IPA,Driftwood Sartori Harvest IPA,Erdinger Weissbier,Estrella Damm (5.4%),Flying Monkeys 12 Minutes to Destiny,Four Winds Nectarous,Great Lakes Brewery Canuck Pale Ale,Guinness Draught,Harp Lager,Heineken,Hoegaarden Grand Cru,Innis & Gunn The Original,Kirin Ichiban,Kona Big Wave Golden Ale,Kozel Cerný (Dark),...,MacKinnon Brothers Red Fox,Maclay's Traditional Pale Ale,Mad Jack Apple Lager,Maisel & Friends Marc's Chocolate Bock,Michelob Ultra,Mickeys Fine Malt Liquor,Midnight Sun M,Mill Street Original Organic Lager,Miller Lite,Molson Carling Light,Moosehead Pale Ale,Muskoka Cream Ale,Muskoka Mad Tom IPA,Newcastle Brown Ale (Non-US Version),Nickel Brook Glory & Gold,NÃ¤rke Kaggen Stormaktsporter,Old Milwaukee,Paulaner Oktoberfest Bier (Wiesn Bier),Peroni,Pilsner Urquell,Rickard's Red,Samuel Smiths Organic Lager,San Miguel,Sapporo Premium Beer / Draft Beer,Shawn & Ed Lagershed Original,Sleeman Clear 2.0,Smithwick's Ale / Draught,Steam Whistle Pilsner,Stella Artois,Superstition Berry White,Tatra Jasne Pe?ne / Pils / Beer,Tiger Beer,Toppling Goliath Kentucky Brunch,Tsingtao Beer Quality Series (Taiwan),Unibroue La Fin du Monde,Unibroue Trois Pistoles,Walkerville Classic Amber Lager,Waterloo IPA,Weihenstephaner Hefeweissbier,Wellington Special Pale Ale SPA
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.590656,0.533279,0.585027,0.646308,0.558765,0.842091,0.366340,0.228563,0.368934,0.398823,0.567207,0.882772,0.390289,0.745123,0.867788,0.384081,0.776089,0.691939,0.312743,0.807819,0.480061,0.663355,0.460624,0.891148,0.619292,0.442776,0.316098,0.726700,0.678457,0.322106,0.353049,0.371822,0.870524,0.920757,0.859908,0.644208,0.639354,0.771235,0.702475,0.660032,...,0.228565,0.361925,0.272459,0.568783,0.838824,0.815005,0.620844,0.428958,0.920785,0.347961,0.504859,0.387118,0.402198,0.889750,0.249793,0.625123,0.748279,0.710168,0.834490,0.831880,0.543679,0.769338,0.635214,0.826580,0.225293,0.413376,0.822692,0.473818,0.858064,0.498278,0.539462,0.707103,0.490992,0.500788,0.818012,0.797564,0.375029,0.330205,0.769786,0.368316
"00cobraR(1,103)",0.640199,0.620231,0.782802,0.707644,0.618547,0.886813,0.493532,0.369488,0.509787,0.541298,0.608215,0.957650,0.515830,0.875736,0.919766,0.454417,0.808137,0.742610,0.479237,0.895887,0.649080,0.752959,0.585577,0.936931,0.723363,0.577834,0.467385,0.810339,0.742775,0.470476,0.486859,0.529342,0.939699,0.960616,0.925451,0.717752,0.734013,0.828613,0.767370,0.704982,...,0.365411,0.614149,0.402461,0.609357,0.917255,0.887385,0.746465,0.607316,1.003138,0.497041,0.610958,0.550841,0.556516,0.938010,0.398347,0.709813,0.848723,0.771143,0.885769,0.898779,0.783351,0.848304,0.661132,0.878483,0.361768,0.576891,0.915732,0.646500,0.921165,0.577885,0.609107,0.778882,0.572008,0.551137,0.944623,0.955733,0.542170,0.494829,0.888235,0.547958
0o9i(439),0.472047,0.416167,0.444973,0.505015,0.420103,0.621618,0.294683,0.163391,0.279565,0.319038,0.478263,0.689583,0.299465,0.653849,0.657492,0.313377,0.572586,0.554608,0.227334,0.670752,0.365499,0.500787,0.377561,0.620179,0.481960,0.345377,0.249833,0.584723,0.528105,0.259443,0.279966,0.287371,0.687322,0.671520,0.673460,0.506771,0.511058,0.596404,0.562793,0.538110,...,0.190348,0.276876,0.197945,0.472872,0.641906,0.608454,0.501616,0.330572,0.699801,0.261346,0.391557,0.295433,0.313734,0.690385,0.193281,0.491966,0.561464,0.570054,0.640408,0.663779,0.408076,0.584204,0.498377,0.599124,0.171367,0.314

In [13]:
# MSE calculates the difference between our predicted and actual ratings
def get_mse(pred, actual):
    # Extract only real movies with ratings
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('Item-based All Nearest Neighbor MSEs:', 
      get_mse(ratings_pred, ratings_matrix.values))

Item-based All Nearest Neighbor MSEs: 3.2201547991273225


## **Top-N based recommendations**



To lower the MSE value further, only the N beers that are most similar to a particular beer should be used in the similarity calculation.

The above method uses the similarity vector of all beers, so the MSE value is rather low.

In [14]:
# Only up to 3 cols. 5 selections with high similarity for 3 beers
top_n_items = [np.argsort(item_sim_df.values[:,3])[:-5:-1]]
top_n_items

[array([ 3, 27, 73, 37])]

In [15]:
# Therefore, only beer with the most similar similarity is used as the similarity vector.
# Applies only to Beer Top_N that has similar similarity to a specific beer -> Takes a long time

def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # Initialize a prediction matrix filled with zeros equal to the size of the user-item rating matrix
    pred = np.zeros(ratings_arr.shape)

    # Loop as many beers in the user-item rating matrix
    for col in range(ratings_arr.shape[1]):
        # Return the indices of n data matrices in the order of similarity in the similarity matrix
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # Personalized prediction score calculation: For each col beer (1 piece), the prediction score of 3015 users
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col,:][top_n_items].dot(
            ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum(item_sim_arr[col,:][top_n_items])

    return pred

ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=10)
print('Item-based nearest TOP-N neighbor MSE: ', 
      get_mse(ratings_pred, ratings_matrix.values))

# The calculated predicted score data is recreated as a DataFrame.
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                  columns=ratings_matrix.columns)

ratings_pred_matrix

Item-based nearest TOP-N neighbor MSE:  1.1103152468291568


beer_name,Hoegaarden Rosée,?ubr,Alexander Keith's India Pale Ale,Baltika 7 Eksportnoe (Export),Bavaria 8.6 (Original),Beck's,Bellwoods Bring Out Your Dead,Bellwoods Jelly King (Raspberry and Blackberry),Berthold Keller Premium Lager,Boxer Ice,BrewDog / Weihenstephan India Pale Weizen,Bud Light,Bud Light Apple,Bud Light Lime,Budweiser,Cantillon Vin Jaune,Carlsberg Pilsner,Chang Beer,Cool Beer Blonde Lager,Corona Extra,Creemore Springs Premium Lager,DAB Original,Dieu du Ciel! Péché Mortel (Bourbon),Dos Equis XX Special Lager,Dragon Stout,Driftwood Fat Tug IPA,Driftwood Sartori Harvest IPA,Erdinger Weissbier,Estrella Damm (5.4%),Flying Monkeys 12 Minutes to Destiny,Four Winds Nectarous,Great Lakes Brewery Canuck Pale Ale,Guinness Draught,Harp Lager,Heineken,Hoegaarden Grand Cru,Innis & Gunn The Original,Kirin Ichiban,Kona Big Wave Golden Ale,Kozel Cerný (Dark),...,MacKinnon Brothers Red Fox,Maclay's Traditional Pale Ale,Mad Jack Apple Lager,Maisel & Friends Marc's Chocolate Bock,Michelob Ultra,Mickeys Fine Malt Liquor,Midnight Sun M,Mill Street Original Organic Lager,Miller Lite,Molson Carling Light,Moosehead Pale Ale,Muskoka Cream Ale,Muskoka Mad Tom IPA,Newcastle Brown Ale (Non-US Version),Nickel Brook Glory & Gold,NÃ¤rke Kaggen Stormaktsporter,Old Milwaukee,Paulaner Oktoberfest Bier (Wiesn Bier),Peroni,Pilsner Urquell,Rickard's Red,Samuel Smiths Organic Lager,San Miguel,Sapporo Premium Beer / Draft Beer,Shawn & Ed Lagershed Original,Sleeman Clear 2.0,Smithwick's Ale / Draught,Steam Whistle Pilsner,Stella Artois,Superstition Berry White,Tatra Jasne Pe?ne / Pils / Beer,Tiger Beer,Toppling Goliath Kentucky Brunch,Tsingtao Beer Quality Series (Taiwan),Unibroue La Fin du Monde,Unibroue Trois Pistoles,Walkerville Classic Amber Lager,Waterloo IPA,Weihenstephaner Hefeweissbier,Wellington Special Pale Ale SPA
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
007lund(92),0.000000,0.245779,0.329653,0.505930,0.141944,2.290314,0.000000,0.000000,0.000000,0.000000,0.000000,2.096753,0.000000,1.226589,2.202730,0.000000,2.217507,0.496644,0.000000,1.987806,0.000000,1.982966,0.286617,2.525622,1.572293,0.000000,0.000000,1.772104,1.270275,0.000000,0.000000,0.000000,2.520638,2.286323,2.110693,0.766681,1.459778,1.992797,1.234976,0.295647,...,0.000000,0.000000,0.000000,0.000000,1.754923,1.233653,0.441541,0.000000,1.859185,0.000000,0.000000,0.000000,0.000000,2.756707,0.000000,0.581618,1.428151,1.672952,1.954839,2.125714,0.000000,1.753344,0.956420,2.357337,0.000000,0.000000,2.156729,0.000000,2.500115,0.127849,0.245913,1.335872,0.043347,0.423742,2.268257,2.170515,0.000000,0.000000,1.930426,0.000000
"00cobraR(1,103)",0.305501,0.206137,1.311189,0.766205,0.000000,1.798044,0.000000,0.000000,0.223849,0.094344,0.279367,1.676296,0.000000,1.393141,1.938457,0.000000,1.827398,0.885150,0.243890,2.083437,0.577420,1.798996,0.594498,1.869895,1.921370,0.000000,0.000000,1.784507,1.251564,0.000000,0.000000,0.000000,2.522900,2.003305,2.129430,1.066011,1.478361,1.868467,1.235396,0.586301,...,0.000000,0.464901,0.000000,0.279123,1.393559,0.837587,0.865311,0.550416,1.439270,0.191909,0.110678,0.202925,0.000000,2.360969,0.000000,1.206904,1.141382,1.685174,1.932064,2.147761,0.947450,1.907989,0.973707,2.088257,0.161274,0.546441,2.016806,0.557320,2.467869,0.314302,0.206249,1.644374,0.238405,0.378540,2.567881,2.583907,0.243754,0.205779,2.590973,0.231588
0o9i(439),0.313758,0.309206,0.230796,0.688903,0.000000,1.710921,0.000000,0.000000,0.000000,0.131142,0.286918,1.490185,0.000000,1.276144,1.848677,0.000000,1.769206,0.885501,0.000000,2.166756,0.000000,1.136321,0.000000,1.198683,1.460573,0.000000,0.000000,1.876643,1.197812,0.000000,0.000000,0.000000,2.235144,1.747442,2.170834,1.209570,1.770059,1.812487,1.507721,0.719426,...,0.000000,0.000000,0.000000,0.286667,1.123276,0.716204,0.438454,0.000000,1.315377,0.000000,0.000000,0.000000,0.000000,2.020792,0.000000,0.650744,0.735247,1.771427,1.962254,2.244347,0.000000,1.683235,1.007860,1.798121,0.000000,0.000

The MSE values ​​have decreased and the prediction matrix values ​​have also changed.

 ## **Recommend to real users**
Given a sample username and rating, we will recommend 3 beers.

As an example, we used user information that already exists in the matrix

In [23]:
# username='snoworsummer(8,581)'
username='lmojoh(3,265)'

# Beer recommendation for specific users
user_rating_id = ratings_matrix.loc[username, :]
# Print the beer that the user ate
user_rating_id[user_rating_id > 0].sort_values(ascending=False)[:10]

beer_name
Guinness Draught                        4.1
Innis & Gunn The Original               3.9
Kronenbourg 1664 Blanc                  3.7
Weihenstephaner Hefeweissbier           3.6
Erdinger Weissbier                      3.4
Newcastle Brown Ale (Non-US Version)    3.3
Kozel Cerný (Dark)                      3.3
Pilsner Urquell                         3.2
Baltika 7 Eksportnoe (Export)           3.1
 Hoegaarden Rosée                       3.1
Name: lmojoh(3,265), dtype: float64

In [24]:
# Recommend a beer that users haven't tasted.
def get_not_tried_beer(ratings_matrix, userId):
    # Extracts all beer information of the user entered as userId and returns it to Series
    # The returned user_rating is a Series object with the userid as an index.
    user_rating = ratings_matrix.loc[userId, :]

    # If user_rating is greater than 0, it is related to the existing movie.
    # Extract target index and make list object
    tried = user_rating[user_rating>0].index.tolist()

    # Make all beer names into list objects
    beer_list = ratings_matrix.columns.tolist()

    # Movies corresponding to tried as a list comprehension are excluded from beer_list
    not_tried = [beer for beer in beer_list if beer not in tried]

    return not_tried

# After extracting the user id index and the beer name entered as not_tried from the predicted rating DataFrame
# Sort by highest predicted rating
def recomm_beer_by_userid(pred_df, userId, not_tried, top_n):
    recomm_beer = pred_df.loc[userId, not_tried].sort_values(ascending=False)[:top_n]
    return recomm_beer

# Extracting the beer name that the user has not eaten
not_tried = get_not_tried_beer(ratings_matrix, username)
not_tried

['?ubr',
 "Alexander Keith's India Pale Ale",
 'Bavaria 8.6 (Original)',
 'Bellwoods Bring Out Your Dead',
 'Bellwoods Jelly King (Raspberry and Blackberry)',
 'Berthold Keller Premium Lager',
 'Boxer Ice',
 'BrewDog / Weihenstephan India Pale Weizen',
 'Bud Light',
 'Bud Light Apple',
 'Bud Light Lime',
 'Cantillon Vin Jaune',
 'Cool Beer Blonde Lager',
 'Creemore Springs Premium Lager',
 'DAB Original',
 'Dieu du Ciel! Péché Mortel (Bourbon)',
 'Dos Equis XX Special Lager',
 'Dragon Stout',
 'Driftwood Fat Tug IPA',
 'Driftwood Sartori Harvest IPA',
 'Estrella Damm (5.4%)',
 'Flying Monkeys 12 Minutes to Destiny',
 'Four Winds Nectarous',
 'Great Lakes Brewery Canuck Pale Ale',
 'Harp Lager',
 'Hoegaarden Grand Cru',
 'Kirin Ichiban',
 'Krombacher Pils',
 'MacKinnon Brothers Red Fox',
 "Maclay's Traditional Pale Ale",
 'Mad Jack Apple Lager',
 "Maisel & Friends Marc's Chocolate Bock",
 'Michelob Ultra',
 'Mickeys Fine Malt Liquor',
 'Midnight Sun M',
 'Mill Street Original Organic La

### Finally, we recommend beer with item-based nearest neighbor CF.

In [25]:
# Only beers similar to top_n are used for recommendations
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=5)

# The calculated predicted score data is recreated as a DataFrame.
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index,
                                   columns=ratings_matrix.columns)

# Extracting the beer name that the user has not eaten
not_tried = get_not_tried_beer(ratings_matrix, username)

# Recommended beer as an item-based nearest neighbor CF
recomm_beer = recomm_beer_by_userid(ratings_pred_matrix, username, not_tried, top_n=3)
recomm_beer = pd.DataFrame(data=recomm_beer.values, index=recomm_beer.index,
                           columns=['Prediction_Score'])
recomm_beer

,Prediction_Score
beer_name,
Stella Artois,2.378123
Hoegaarden Grand Cru,2.230628
Krombacher Pils,2.098706
